#**PLAYGROUND V2.5 LORA CREATOR - Adapted from SD XL Lora training by linoytsaban*

# Install necessary libs.

In [ ]:
!pip install huggingface_hub datasets pillow xformers bitsandbytes transformers accelerate wandb dadaptation prodigyopt torch -q


In [ ]:
!pip install peft -q

In [ ]:
!pip install git+https://github.com/huggingface/diffusers.git -q

Downloading the script!

In [ ]:
!wget https://raw.githubusercontent.com/huggingface/diffusers/main/examples/advanced_diffusion_training/train_dreambooth_lora_sdxl_advanced.py

In [ ]:
!wget https://gist.githubusercontent.com/artificialguybr/a1c58ad578d0446d493c9793093196e1/raw/b8cede5d9ad5f9911de70ac187c6b078eea693c8/sd-dataset-to-hf-dataset.py

In [ ]:
!accelerate config default

Login with your HuggingFace Account!

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

Convert your currently SD XL (Kohya) style Dataset to HF Dataset.

In [ ]:
# Required imports
import os
from huggingface_hub import HfApi, login
from datasets import Dataset, Features, Value, Image
from PIL import Image as PILImage
import io

## Configuration
# Define the directory where your images and texts are stored
data_folder = ""  # Example: "/content/drive/MyDrive/dataset"

# Name your dataset as desired
dataset_name = ""  # Example: "my-first-dataset"
dataset_repo_id = ""  # Example: "your_username/my-first-dataset"
hf_token = ""  # Your Hugging Face token

## Initialize the Hugging Face API
api = HfApi()
login(hf_token)

## Define the dataset features
features = Features({
    "image": Image(),
    "text": Value("string"),
})

## Create a dictionary to store the corresponding images and texts
data = {"image": [], "text": []}

## Accepted image file extensions
image_extensions = [".png", ".jpg", ".jpeg", ".gif"]

## Iterate through image files
for filename in os.listdir(data_folder):
    # Check if the file has an accepted image extension
    if any(filename.lower().endswith(ext) for ext in image_extensions):
        image_path = os.path.join(data_folder, filename)
        txt_filename = os.path.splitext(filename)[0] + ".txt"
        txt_path = os.path.join(data_folder, txt_filename)

        if os.path.exists(txt_path):
            # Read the content of the text file
            with open(txt_path, "r") as txt_file:
                text = txt_file.read()

            # Load the image using the pillow library
            with PILImage.open(image_path) as image:
                # Convert the image to the format expected by Hugging Face Datasets
                image_bytes = io.BytesIO()
                image.save(image_bytes, format='PNG')
                image_bytes = image_bytes.getvalue()

            # Add the image and text to the dictionary
            data["image"].append(image_bytes)
            data["text"].append(text)

## Create the dataset on Hugging Face
dataset = Dataset.from_dict(data, features=features)
dataset.push_to_hub(dataset_repo_id, private=True)  # Set private=False if you want the dataset to be public

print("Dataset successfully uploaded to Hugging Face!")


In [ ]:
!pip install python-slugify
from slugify import slugify

model_name = "3d icon SDXL LoRA" # @param
output_dir = slugify(model_name)

# **Instance & Validation Prompt**
* `instance_prompt` -
    * when custom captions are enabled this prompt is still used in case there are missing captions, as well as in the model's readme.
    * If custom captions are not used, this prompt will be used as the caption for all training images.
* `validation_prompt` -
    * this prompt is used to generate images throught the training process, this way you can see the models learning curve during training.
    * you can also change `num_validation_images` (4 by default) and `validation_epochs` (50 by default) to control the amount images generated with the validation prompt, and the number of ephochs between each dreambooth validation.  

In [ ]:
instance_prompt = "3d icon in the style of TOK" # @param
validation_prompt = "a TOK icon of an astronaut riding a horse, in the style of TOK" # @param

**Set your LoRA rank**
The rank of your LoRA is linked to its expressiveness.
The bigger the rank the closer we are to regular dreambooth, and in theory we have more expressive power (and heavier weights).

For a very simple concept that you have a good high quality image set for (e.g. a pet, a generic object), a rank as low as 4 can be enough to get great results. We reccomend going between 8 and 64 depending on your concept and how much of a priortiy it is for you to keep the LoRA small or not.  

In [ ]:
rank = 8 # @param

#Dataset Name/Local/HF

In [ ]:
dataset_name=./3d_icon # @param
caption_column=prompt # @param

In [ ]:
#!/usr/bin/env bash
!accelerate launch train_dreambooth_lora_sdxl_advanced.py \
  --pretrained_model_name_or_path="playgroundai/playground-v2.5-1024px-aesthetic" \
  --dataset_name="$dataset_name" \
  --instance_prompt="$instance_prompt" \
  --validation_prompt="$validation_prompt" \
  --output_dir="$output_dir" \
  --caption_column="$caption_column" \
  --do_edm_style_training \
  --mixed_precision="bf16" \
  --resolution=1024 \
  --train_batch_size=3 \
  --repeats=1 \
  --report_to="wandb"\
  --gradient_accumulation_steps=1 \
  --gradient_checkpointing \
  --learning_rate=1e-5 \
  --optimizer="AdamW"\
  --lr_scheduler="constant" \
  --rank="$rank" \
  --max_train_steps=2000 \
  --checkpointing_steps=2000 \
  --seed="0" \
  --push_to_hub

In [ ]:
from huggingface_hub import whoami
from pathlib import Path
from IPython.display import display, Markdown

username = whoami(token=Path("/root/.cache/huggingface/"))["name"]
repo_id = f"{username}/{output_dir}"

link_to_model = f"https://huggingface.co/{repo_id}"

display(Markdown("### Your model has finished training.\nAccess it here: {}".format(link_to_model)))

#LOAD INFERENCE

In [ ]:
import torch
from huggingface_hub import hf_hub_download, upload_file
from diffusers import DiffusionPipeline
from diffusers.models import AutoencoderKL
from safetensors.torch import load_file

pipe = DiffusionPipeline.from_pretrained(
    "playgroundai/playground-v2.5-1024px-aesthetic",
    torch_dtype=torch.float16,
    variant="fp16",
).to("cuda")

pipe.load_lora_weights(repo_id, weight_name="pytorch_lora_weights.safetensors")
